In [28]:
from bs4 import BeautifulSoup
import json
import re
import requests
import pandas as pd
from urllib.parse import urlparse

In [29]:
url_base='https://www.ntucc.gov.tw'
url_work='/ntucc/Recruit.action'
url_full=url_base+url_work
g=requests.get(url_full)
soup=BeautifulSoup(g.content, 'html.parser')

In [30]:
print(g.cookies['JSESSIONID'])

817BDA5CA13382FCC6E16CA684AE3EC6


In [31]:
my_params = {'jobtype': '-1', 'q_seekValue': '','pagenum':'1'}

In [32]:
ntucc_work_data = json.loads(requests.get('https://www.ntucc.gov.tw/ntucc/RecruitAjax.action',params=my_params, cookies = g.cookies).content)

In [24]:
data.keys()

dict_keys(['id', 'jobtype', 'msg', 'pk', 'q_seekValue', 'queryList', 'totalPages', 'visiblePages'])

# Get Pages link

In [26]:
totalcount = ntucc_work_data['totalPages']
totalPages = ntucc_work_data['totalPages']

# Get work table

In [40]:
for i, item in enumerate(ntucc_work_data['queryList']):
    title = item['title']
    origantion = item['jobDepnm']
    begin_date = item['adate_sh']
    dead_line = item['edate_sh']

    file_list = []
    for _file in item['recruitFile']:
        file_list.append([_file['fileName'],'https://www.ntucc.gov.tw/ntucc/RecruitAjax!download.action?id='+_file['id']])
    #if item['recruitNo']:
    #    link = 'https://reg.ntuh.gov.tw/WebApplication/Administration/NtuhGeneralSelect/Entry.aspx?selectno='+item['recruitNo']
    #else:
    #    link = url_base+item['ctx'].lstrip('<p><a href="../').split('"><span ')[0]
    print('#{}召聘職稱: {} 召聘單位: {}\n 期限: {}\n 連結：{}'.format(i+1, title, origantion, dead_line, file_list))
    #work_table.append([title, origantion, dead_line, link])


#1召聘職稱: 人事室 副管理師或管理師(職務代理人)（預估缺） 召聘單位: 人事室
 期限: 2022/07/13
 連結：[['癌醫中心醫院(副)管理師(人事室職務代理人1名)1110706', 'https://www.ntucc.gov.tw/ntucc/RecruitAjax!download.action?id=2c9a7cf97fde99380181a412800f0200'], ['台大癌醫中心分院各類人員甄選登記表-(修正版、職前經歷查驗)', 'https://www.ntucc.gov.tw/ntucc/RecruitAjax!download.action?id=2c9a7cf97fde99380181a41a0fa40201'], ['台大癌醫中心分院各類人員甄選登記表-(修正版、職前經歷查驗)', 'https://www.ntucc.gov.tw/ntucc/RecruitAjax!download.action?id=2c9a7cf97fde99380181a41a46e00202'], ['台大癌醫中心分院各類人員甄選登記表-(修正版、職前經歷查驗)', 'https://www.ntucc.gov.tw/ntucc/RecruitAjax!download.action?id=2c9a7cf97fde99380181a41a7bca0203'], ['信封封面(直式-各類人員甄選-癌醫)1100820', 'https://www.ntucc.gov.tw/ntucc/RecruitAjax!download.action?id=2c9a7cf97fde99380181a41ac1820204']]
#2召聘職稱: 藥劑部 服務員 召聘單位: 藥劑部
 期限: 2022/07/28
 連結：[['癌醫中心醫院服務員(1名)(藥劑部)1110802', 'https://www.ntucc.gov.tw/ntucc/RecruitAjax!download.action?id=2c9a7cf97fde99380181ad51ff52020d'], ['台大癌醫中心分院各類人員甄選登記表-(修正版、職前經歷查驗)', 'https://www.ntucc.gov.tw/ntucc/RecruitAjax!download.action

In [5]:
def get_each_page_wrok_table(soup,work_table):
    for work in soup.find_all('table',class_="table table-striped table-hover news_table")[0].find("tbody").find_all("tr"):
        all_td = work.find_all("td")
        title = all_td[0].text
        place = '察看連結的簡章'
        deadline = all_td[2].text
        
        link=url_base+all_td[3].find('a').get('href')
        if "錄取名單" in title:
            print('skip: ',title)
        elif "甄試名單" in title:
            print('skip: ',title)
        elif "複試名單" in title:
            print('skip: ',title)
        elif "志願服務工作" in title:
            print('skip: ',title)
        elif "甄審會" in title:
            print('skip: ',title)
        elif "時薪人員" in title:
            print('skip: ',title)
        else:
            originization = re.search("\B院((.*)[室,部,中心])",title).group(1)
            new_title = re.search("\B聘((.*)[員,師,廚])",title).group(1)
            print("召聘職稱",new_title,'院區',place,"截止日期",deadline,"職缺單位",originization,"links",link)
            work_table.append([new_title, deadline, originization, place, link ])
    return work_table

In [6]:
def get_base_web_data(url_base,url_work_table=None):
    if url_work_table != None:
        url = url_base + url_work_table
    else:
        url = url_base
    g=requests.get(url)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

In [8]:
work_table = []
for page in pages_link:
    soup = get_base_web_data(page)
    work_table = get_each_page_wrok_table(soup,work_table)

skip:  公告本院護理部111年5月6日公開甄選院聘護理師/實習護士儲備30名錄取名單
skip:  公告本院護理部111年5月4日公開甄選院聘護理師/實習護士儲備30名錄取名單
召聘職稱 助廚 院區 察看連結的簡章 截止日期 2022-12-31 職缺單位 營養室 links https://www.ylh.gov.tw/public/jobnews/300f01ae03a2571529b5b1f21e968326.pdf
召聘職稱 照顧服務員 院區 察看連結的簡章 截止日期 2022-12-31 職缺單位 護理部 links https://www.ylh.gov.tw/public/jobnews/2a78c957732fe3c5a6dc5791a081da3c.pdf
召聘職稱 助理管理師 院區 察看連結的簡章 截止日期 2022-06-23 職缺單位 安全衛生室111年5月4日公開甄選院聘助理管理師(身心 links https://www.ylh.gov.tw/public/jobnews/075d3c7f9f84898c8f932d7340d6ca11.pdf
召聘職稱 管理師 院區 察看連結的簡章 截止日期 2022-06-10 職缺單位 企劃管理室 links https://www.ylh.gov.tw/public/jobnews/753fdd4238b4b67158f1d5c504a2db4e.pdf
skip:  公告本院111/5/4甄審會公開甄選複試結果(公職醫事人員類)
skip:  公告本院護理部(病房)111年4月26日公開甄選院聘助理管理師1名「錄取名單」
skip:  公告本院檢驗醫學部111年5月5日公開甄選院聘助理管理師職務代理人1名「錄取名單」
skip:  公告本院護理部(護理之家)111年4月28日公開甄選院聘照顧服務員1名「錄取名單」
skip:  公告本院護理部(門診)111年4月22日甄選院聘護理師2名錄取名單
skip:  公告本院護理部111年5月3日公開甄選院聘護理師/實習護士儲備30名錄取名單
skip:  公告本院護理部111年4月26日公開甄選院聘護理師/實習護士儲備30名錄取名單
skip:  公告本院護理部111年4月22日公開甄選院聘護理師/實習護士儲備30名錄取名單
skip:  公告本

In [34]:
ntucc_work_data['queryList'][0]

{'FILEEXT1': None,
 'FILENAME1': None,
 'FILEPATH1': None,
 'FILEREMARK1': None,
 'FILESIZE1': None,
 'SEQ1': None,
 'TITLE1': None,
 'TYPENO1': None,
 'adate': '2022-07-06T00:00:00',
 'adate_ch': '111/07/06',
 'adate_sh': '2022/07/06',
 'attachNum1': 0,
 'attachNum2': 0,
 'attachNum3': 0,
 'attachNum4': 0,
 'attachNum5': 0,
 'attachNum6': 0,
 'createDate': '2022-06-27T15:09:01',
 'createUser': 'HR',
 'depno': 'ntucc',
 'edate': '2022-07-13T00:00:00',
 'edate_ch': '111/07/13',
 'edate_sh': '2022/07/13',
 'id': '2c9a7cf97fde99380181a3fe00f501ff',
 'isuse': '1',
 'item1list': None,
 'jobDepnm': '人事室',
 'jobDepno': '19',
 'jobtype': '3',
 'lang': '1',
 'mid': 4,
 'modifyDate': '2022-07-06T13:25:09',
 'modifyUser': 'HR',
 'odate': '2022-10-31T00:00:00',
 'recruitFile': [{'FILEID_A': None,
   'FILE_A': None,
   'FILE_AContentType': None,
   'FILE_AFileName': None,
   'ID_A': None,
   'REMARK_A': None,
   'SEQ_A': None,
   'TYPE_A': None,
   'createDate': '2022-06-27T15:31:24',
   'createUse

In [110]:
re.search("\B聘((.*)師)","公告本院護理部公開甄選院聘護理師(手術室)儲備").group(1)

'護理師'

In [112]:
work_table=pd.DataFrame(work_table, columns=['召聘職稱','期限' ,"職缺單位" , '院區','連結'])

print(work_table)

        召聘職稱          期限               職缺單位       院區  \
0        護理師  2022-07-31   護理部公開甄選院聘護理師(手術室  察看連結的簡章   
1        護理師  2022-07-31                護理部  察看連結的簡章   
2     契約主治醫師  2022-05-20           環境及職業醫學部  察看連結的簡章   
3      助理管理師  2022-05-09  主計室公開甄選院聘助理管理師(身心  察看連結的簡章   
4       副管理師  2022-05-10                護理部  察看連結的簡章   
5      醫事檢驗師  2022-05-20                病理部  察看連結的簡章   
6        佐理員  2022-05-09            護理部(供應室  察看連結的簡章   
7         助廚  2022-12-31                營養室  察看連結的簡章   
8        護理師  2022-07-31   護理部公開甄選院聘護理師(手術室  察看連結的簡章   
9        護理師  2022-07-31                護理部  察看連結的簡章   
10    契約主治醫師  2022-05-20           環境及職業醫學部  察看連結的簡章   
11     助理管理師  2022-05-09  主計室公開甄選院聘助理管理師(身心  察看連結的簡章   
12      副管理師  2022-05-10                護理部  察看連結的簡章   
13     醫事檢驗師  2022-05-20                病理部  察看連結的簡章   
14       佐理員  2022-05-09            護理部(供應室  察看連結的簡章   
15  資深/專科護理師  2022-06-15            醫療部/護理部  察看連結的簡章   

                                               

In [ ]:
https://www.ntucc.gov.tw/ntucc/RecruitAjax!download.action?id=2c9a7cfa7fde98350181c6e991d60268